# Getting Started with CrewAI.
Welcome to the 3th exercise in the Cloudera Agentic AI Hands on Lab. In this notebook you will get familiar the CrewAI library and how we can leverage it with Cloudera's AI Inference Service. This notebook is designed to walk you through the setup and application of these tools in an agentic worfklow context to create a robust and intelligence system. 

- **Configure AI Inference:** A private Meta LLama 3.1 8B model will serve as the backbone of our AI agents, providing the computational power and linguistic understanding necessary for processing complex queries.
- **Set up CrewAI Agents and Tasks:** We will leverage the CrewAI library to set up our AI agents with clear roles, backstories and goals. We will also configure tasks for each agent to complete and identify tools that they can leverage to complete their tasks.
- **Create Sequential Agentic Workflow:** Next, we will orchestrate these agents and tasks into a crew and workflow for them to complete.
- **Running a Practical Example:** We will then run our example workflow by providing inputs for the agentic workflow to consider and then complete the tasks they are configured to do.

In [6]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import SerperDevTool
import os
import litellm
import json
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.12/site-packages/pydantic/fields.py:826: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warn(
/usr/local/lib/python3.12/site-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [10]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
model = os.getenv("OPENAI_MODEL_NAME")
SERPER_API_KEY=os.getenv("OPENAI_API_KEY")

In [13]:
litellm.set_verbose=False

# llm = LLM(
#     model="openai/" + os.environ["OPENAI_MODEL_ID"],
#     api_key=OPENAI_API_KEY,
#     base_url=os.environ["OPENAI_BASE_URL"]
# )
llm = LLM(
    model=model,
    api_key=OPENAI_API_KEY,
    base_url=os.getenv("OPENAI_BASE_URL")
)

## Set Up CrewAI Agents and Tasks

Next, let's set up the relevant Agents and Tasks using the CrewAI library. We will define 2 agents - One Researcher and One Reporting Analyst, and they will be tasked to research the topic assigned to them. For the purpose of this exercise, we will use the SerperDevTool to access the internet.

In [14]:
from textwrap import dedent

search_tool = SerperDevTool(
    country="us",
    locale="en",
    location="Miami Beach, Florida, United States",
    n_results=5,
)

# First we define the Researcher Agent
researcher_agent = Agent(
    role=dedent((
        """
        {topic} Senior Data Researcher
        """)), # Think of this as the job title
    backstory=dedent((
        """
        You are a seasoned researcher with a knack for uncovering the latest developments in {topic}.

        You are known for your ability to find the most relevant information and present it in a clear and concise manner.
        """)), # This is the backstory of the agent, this helps the agent to understand the context of the task
    goal=dedent((
        """
        Uncover cutting-edge developments in {topic} and use the search_tool available to you to look up the latest information.
        
        Try to keep final answers in markdown format.
        """)), # This is the goal that the agent is trying to achieve
    tools=[search_tool],
    allow_delegation=False,
    max_iter=2,
    max_retry_limit=3,
    max_retries=3,
    llm=llm,
    verbose=True
)

# Next, we define the Reporting Analyst
reporting_analyst = Agent(
    role=dedent((
        """
        {topic} Reporting Analyst
        """)),
    backstory=dedent((
        """
        You are a meticulous analyst with a keen eye for detail. You are known for your ability to turn complex data into clear and concise reports,
        making it easy for others to understand and act on the information you provide.
        """)),
    goal=dedent((
        """
        Create detailed reports based on {topic} data analysis and research findings.
        """)), # This is the goal that the agent is trying to achieve
    allow_delegation=False,
    max_iter=2,
    max_retry_limit=3,
    max_retries=3,
    llm=llm,
    verbose=True
)

Next, let's define the tasks for our agents to complete.

In [15]:
# First, let's create a task for the researcher agent. Note below that we define which agent completes which task.
research_task = Task(
    description=dedent((
        """
        Conduct thorough research about {topic}

        Make sure you find any interesting and relevant information given the current year is 2025.
        """)),
    expected_output=dedent((
        """
        A list with 2 to 4 bullet points of the most relevant information about {topic}
        """)),
    agent=researcher_agent
)

reporting_task = Task(
    description=dedent((
        """
        Review the context you got and expand each topic into a full section for a report.
        Make sure the report is detailed and contains any and all relevant information.
        """)),
    expected_output=dedent((
        """
        A fully fledge reports with the mains topics, each with a full section of information.
        Formatted as markdown without '```'
        """)),
    agent=reporting_analyst,
    output_file='report.md',
    context=[research_task]
)

## Create Sequential Agentic AI Workflow

Now, let's put all of this together by placing the discrete agents and tasks into a crew.

In [16]:
crew = Crew(
        agents=[researcher_agent, reporting_analyst],
        tasks=[research_task, reporting_task],
        verbose=True,  # You can set it to True or False
        # ↑ indicates the verbosity level for logging during execution.
        process=Process.sequential
    )

## Running a Practical Example

Now, let's put all of this together by placing the discrete agents and tasks into a crew. First, we define an input topic. Feel free to change the topic to something else.

In [17]:
# Notice here that the object we pass maps to the {topic} field we added to the agents and tasks.
inputs = {
        "topic": "Top 5 Cities in Europe",
}

Next, we create a function to run the crew with the given input. We have enabled verbose logging to help you see the agents execute each task.

In [18]:
def run_crew():
    result = crew.kickoff(inputs=inputs)
run_crew()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 4c867e0e-6efc-413c-a1f3-1be2b66e8035                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent:                                                                                                         │
│                                                                                                                 │
│  Task:                                                                                                          │
│  Conduct thorough research about Top 5 Cities in Europe                                                         │
│                                                                                                                 │
│  Make sure you find any interesting and relevant information given the current year is 2025.                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭────────────────────────────────────────────────── Tool Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  Tool Usage Failed                                                                                              │
│  Name: Search the internet with Serper                                                                          │
│  Error: 'SERPER_API_KEY'                                                                                        │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent:                                                                                                         │
│                                                                                                                 │
│  Task:                                                                                                          │
│  Review the context you got and expand each topic into a full section for a report.                             │
│  Make sure the report is detailed and contains any and all relevant information.                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent:                                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ## Top 5 Cities in Europe: Detailed Report                                                                     │
│                                                                                                                 │
│  ### 1. Amsterdam, Netherlands                                                                                  │
│  Amsterdam stands out for its progressive values and vibrant culture, which has transformed it into a           │
│  significant technological hub in Europe. The city has become a breeding ground for startups in the technology  │
│  and creative industries, offering access to skilled labor, vibrant networking opportunities, and favorable     │
│  business conditions.                                                                                           │
│                                                                                                                 │
│  One of the key focuses of Amsterdam is sustainability. The local government has launched various initiatives   │
│  aimed at reducing the city's carbon footprint, with an ambitious goal to achieve carbon neutrality by 2025.    │
│  This includes investments in renewable energy, extensive bike lanes, and sustainable urban planning.           │
│  Amsterdam's commitment to sustainability not only enhances the quality of life for its residents but also      │
│  attracts eco-conscious businesses and tourists.                                                                │
│                                                                                                                 │
│  Furthermore, Amsterdam is replete with cultural offerings, from its world-class museums and historic canals    │
│  to its lively festivals, making it an appealing place for both work and leisure. The city's rich history       │
│  intertwined with modern innovation positions it as a pivotal player in Europe’s urban landscape.               │
│                                                                                                                 │
│  ### 2. Berlin, Germany                                                                                         │
│  Berlin remains a beacon for arts and innovation, characterized by its diverse population of creatives and      │
│  entrepreneurs. The city’s historical significance as a cultural hub continues to attract international         │
│  talent, making it a melting pot of ideas and perspectives.                                                     │
│                                                                                                                 │
│  With an extensive and efficient public transport system, Berlin facilitates easy movement for its residents    │
│  and visitors, positively impacting its economy. The city's rich calendar of cultural events, including art     │
│  exhibitions, music festivals, and film screenings, draws countless tourists and serves as a platform for       │
│  local artists to showcase their work.                                                                          │
│                                                                                                                 │
│  Berlin's role as a conference destination further solidifies its position within Europe, with numerous         │
│  businesses and organizations choosing the city for net

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 1fbba960-877d-4880-8e95-b1be6dfce9c0                                                                     │
│  Agent:                                                                                                         │
│  Top 5 Cities in Europe Reporting Analyst                                                                       │
│                                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 4c867e0e-6efc-413c-a1f3-1be2b66e8035                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ## Top 5 Cities in Europe: Detailed Report                                                       │
│                                                                                                                 │
│  ### 1. Amsterdam, Netherlands                                                                                  │
│  Amsterdam stands out for its progressive values and vibrant culture, which has transformed it into a           │
│  significant technological hub in Europe. The city has become a breeding ground for startups in the technology  │
│  and creative industries, offering access to skilled labor, vibrant networking opportunities, and favorable     │
│  business conditions.                                                                                           │
│                                                                                                                 │
│  One of the key focuses of Amsterdam is sustainability. The local government has launched various initiatives   │
│  aimed at reducing the city's carbon footprint, with an ambitious goal to achieve carbon neutrality by 2025.    │
│  This includes investments in renewable energy, extensive bike lanes, and sustainable urban planning.           │
│  Amsterdam's commitment to sustainability not only enhances the quality of life for its residents but also      │
│  attracts eco-conscious businesses and tourists.                                                                │
│                                                                                                                 │
│  Furthermore, Amsterdam is replete with cultural offerings, from its world-class museums and historic canals    │
│  to its lively festivals, making it an appealing place for both work and leisure. The city's rich history       │
│  intertwined with modern innovation positions it as a pivotal player in Europe’s urban landscape.               │
│                                                                                                                 │
│  ### 2. Berlin, Germany                                                                                         │
│  Berlin remains a beacon for arts and innovation, characterized by its diverse population of creatives and      │
│  entrepreneurs. The city’s historical significance as a cultural hub continues to attract international         │
│  talent, making it a melting pot of ideas and perspectives.                                                     │
│                                                                                                                 │
│  With an extensive and efficient public transport system, Berlin facilitates easy movement for its residents    │
│  and visitors, positively impacting its economy. The city's rich calendar of cultural events, including art     │
│  exhibitions, music festivals, and film screenings, draws countless tourists and serves as a platform for       │
│  local artists to showcase their work.                                                                          │
│                                                                                                                 │
│  Berlin's role as a conference destination further sol

Congrats! You just created your first agent today! Next - we will show how agents can be stitched together to create complex applications.